### Import Library and load dataset

In [4]:
ls

addiction_2019_features_tfidf_256.csv  sample_data/


In [3]:
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras_nlp
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd



In [6]:
Data = pd.read_csv('addiction_2019_features_tfidf_256.csv', usecols = ['post'] )


Data.head()

,post
0,Quitting coke and need advice So I’m 20. I’ve ...
1,I am addicted to sleep I really really like to...
2,Addicted to scratching my balls This may sound...
3,Who else has lost their kids because their add...
4,Please Read I’ve been suffering addiction to k...


In [7]:
Data.info() # 1257 dataset of depression post on, no NAN values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1257 entries, 0 to 1256
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   post    1257 non-null   object
dtypes: object(1)
memory usage: 9.9+ KB


In [8]:
# Code to remove digits and lower the text. punctuations left as it is important in text generation and comprehension
import re

def process(text):
  text = text.lower()
  text = re.sub(r'\d+', '', text)
  return text



Data['post'] = Data['post'].apply(process)

Data

,post
0,quitting coke and need advice so i’m . i’ve be...
1,i am addicted to sleep i really really like to...
2,addicted to scratching my balls this may sound...
3,who else has lost their kids because their add...
4,please read i’ve been suffering addiction to k...
...,...
1252,anyone relate i have relapsed a lot even after...
1253,will my husband ever stop using coke? my husba...
1254,im and cant stop stealing first i just want t...
1255,"heroin overdose, medical doctor advice? i had ..."


In [9]:
# convert dataframe to tf dataset
train_dataset = tf.data.Dataset.from_tensor_slices((Data['post'].values))

for text in train_dataset.take(5):
    print(text.numpy().decode('utf-8'))




quitting coke and need advice so i’m . i’ve been thinking for a while about quitting coke but have been getting sucked into the whole “oh it’s just one line” thing and then buying a whole sack and the cycle continues. i don’t know what to do. coke is all around me. most people i interact with daily are heavy users, i seriously can’t escape it. all of my roommates use heavily and regularly, most of my coworkers use it, and so do most of my friends. another issue is the alcoholic culture around me. i’ll drink and then buy coke, or drink and do some if offered because i’m drunk and don’t care. and the thing is drinking is so heavy in my social circle too, and where there’s alcohol there’s coke. i don’t know, i’m just afraid if i drink i’ll do coke, but also don’t want to stop drinking altogether. i would just like to hear from people who have been through this and what helped you. 
i am addicted to sleep i really really like to sleep. my family thinks i am depressed or high all the time b

### importing the GPT2 model and generating text relating to depression

In [25]:
preprocessor = keras_nlp.models.GPT2CausalLMPreprocessor.from_preset('gpt2_base_en', sequence_length = 256)

gpt2 = keras_nlp.models.GPT2CausalLM.from_preset('gpt2_base_en', preprocessor = preprocessor)

gpt2.summary()

Preprocessor: "gpt2_causal_lm_preprocessor_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gpt2_tokenizer (GPT2Tokenizer)                                │                       Vocab size: 50,257 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gpt2_causal_lm_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gpt2_backbone (GPT2Backbone)  │ (None, None, 768)         │     124,439,808 │ padding_mask[0][0],        │
│                               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 50257)       │      38,597,376 │ gpt2_backbone[0][0]        │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 124,439,808 (474.70 MB)

 Trainable params: 124,439,808 (474.70 MB)

 Non-trainable params: 0 (0.00 B)

In [26]:
import time
start_time = time.time()

statement = gpt2.generate('i am a drug addict, i need to get help', max_length = 150)

print('\nstatememnt :')
print(statement)

end_time = time.time()

print( end_time - start_time)




statememnt :
i am a drug addict, i need to get help and help with my addiction

i am a drug addict, i need to get help and help with my addiction

i am a drug addict, i need to get help and help with my addiction

i am a drug addict, i need to get help and help with my addiction

i am a drug addict, i need to get help and help with my addiction

i am a drug addict, i need to get help and help with my addiction

i am a drug addict, i need to get help and help with my addiction

i am a drug addict, i need to get help and help with my addiction

i am a drug addict
13.137445211410522


In [27]:
import time
start_time = time.time()

statement = gpt2.generate('i will like to travel to Australia', max_length = 150)

print('\nstatememnt :')
print(statement)

end_time = time.time()

print( end_time - start_time)


statememnt :
i will like to travel to Australia for my family to celebrate my birthday. i will also like to go to a concert and see my friends in a different country, and to see my favourite music and movies. i am also going to visit the local theatre where i will be watching some of the biggest bands and bands in the city. i will be going out to see some of my favourite people, and to see some of my favourite places to go.

i am so excited for this trip and i am so grateful that you guys will be supporting me and helping me make this happen. i have been so busy so far and i need to go and see all the amazing bands that will be performing at the concert. i am so excited
1.3608176708221436


### Trainig GPT 2 model on the depression dataset. the tokenisation and embeding handled by the model. it is computational expensive so i trained for  2 epoch

In [12]:
num_epochs = 2
train_dataset = train_dataset.batch(32).cache().prefetch(tf.data.AUTOTUNE)

lr = keras.optimizers.schedules.PolynomialDecay(5e-5,
                                                decay_steps = train_dataset.cardinality() * num_epochs,
                                                end_learning_rate = 0.0)

loss_func = keras.losses.SparseCategoricalCrossentropy(from_logits = True)

gpt2.compile(optimizer = keras.optimizers.Adam(lr),
             loss = loss_func,
             weighted_metrics = ['accuracy'])


gpt2.fit(train_dataset, epochs = num_epochs)

Epoch 1/2
40/40 ━━━━━━━━━━━━━━━━━━━━ 207s 3s/step - accuracy: 0.3152 - loss: 2.4149
Epoch 2/2
40/40 ━━━━━━━━━━━━━━━━━━━━ 77s 2s/step - accuracy: 0.3457 - loss: 2.2059


In [14]:
import time
start_time = time.time()

statement = gpt2.generate('i am a drug addict, i need to get help', max_length = 150)

print('\nstatememnt :')
print(statement)

end_time = time.time()

print( end_time - start_time)


statememnt :
i am a drug addict, i need to get help. i am addicted to cocaine, heroin, meth, and prescription drugs. 

i am addicted to the addictive nature of my life. i am addicted to the fact that i can do anything. 

i am not sure how to get through life without a drug. 
12.848846435546875


In [15]:
import time
start_time = time.time()

statement = gpt2.generate('i am an addict, i need to see a Doctor', max_length = 150)

print('\nstatememnt :')
print(statement)

end_time = time.time()

print( end_time - start_time)


statememnt :
i am an addict, i need to see a Doctor, i am addicted to drugs and alcohol. i need to be sober. i want to be free of this addiction. i want to be free from it. i want to be able to live my life without the addiction. i need to be free from this addiction. 

i am an addict, i need to see a Doctor, i am addicted to drugs and alcohol. i need to be clean. i want to be free from this addiction. i want to be able to live my life without the addiction. 
i am an addict, i need to see a Doctor, i am addicted to drugs and alcohol. i need to be clean. i want to be free
1.3170340061187744


In [22]:
# Assuming `model` is your trained Keras model
gpt2.save("/content/drive/MyDrive/gpt2_dep.keras")  # Save the entire model to an HDF5 file


#### Observation
The pretrained GPT2 model alone did not do so well on this type of question related to depression, but when trained for just 2 epoch, it was able to generate more reasonble response.
Noted on general questions like ' i will like to travel to Australia', the pretrained model output a better reply